In [17]:
import cv2
import os
import pandas as pd
import numpy as np
import pickle as pkl


In [18]:
folder_path = '../data/'

In [19]:
def store_images_path(folder):
    df = pd.DataFrame(columns=['Path','Identity','Facial Expression','age'])
    for file in os.listdir(folder):
        features = file.split('_')
        age = 0
        if features[0] == 'haziq':
            age = 21
        else:
            age = 26 
        df = df.append({'Path':os.path.join(folder,file),'Identity':features[0],'Facial Expression':features[1][0],'age':age},ignore_index=True)
    return df

In [20]:
df = store_images_path(folder=folder_path)

In [21]:
df

,Path,Identity,Facial Expression,age
0,../data/haziq_0 (31).jpeg,haziq,0,21
1,../data/haziq_0 (32).jpeg,haziq,0,21
2,../data/haziq_0 (33).jpeg,haziq,0,21
3,../data/haziq_0 (34).jpeg,haziq,0,21
4,../data/haziq_0 (35).jpeg,haziq,0,21
...,...,...,...,...
56,../data/sulaiman_1 (38).jpeg,sulaiman,1,26
57,../data/sulaiman_1 (39).jpeg,sulaiman,1,26
58,../data/sulaiman_1 (40).jpeg,sulaiman,1,26
59,../data/sulaiman_1 (41).jpeg,sulaiman,1,26


In [22]:
df = df.sample(frac = 1)

In [23]:
df.to_csv('Data.csv',index=False)

In [24]:
def load_image_and_resize_vector(df):
    images = []
    for path in df['Path']:
        img = cv2.imread(path)
        resize_image = cv2.resize(img, (224,224))
        images.append(resize_image)
    return np.array(images)
        

In [25]:
def Compute_histogram(img):
   blue_color = cv2.calcHist([img], [0], None, [256], [0, 256])
   red_color = cv2.calcHist([img], [1], None, [256], [0, 256])
   green_color = cv2.calcHist([img], [2], None, [256], [0, 256])
   list = blue_color.flatten() 
   list = np.concatenate((list, red_color.flatten()))
   list = np.concatenate((list, green_color.flatten()))
   
   return list


In [26]:
def Compute_3x3_vector(img):
    height,width = img.shape[0],img.shape[1]
    row1 = round(height/3)
    row2 = round(height*2/3)
    row3 = height-row2
    col1 = round(width/3)
    col2 = round(width*2/3)
    col3 = width-col2

    img1 = img[:row1,:col1]
    feature_vector = Compute_histogram(img1)
    img12 = img[:row1,col1:col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img12)))
    img13 = img[:row1,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img13)))

    img2 = img[row1:row2,:col1]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img2)))
    img22 = img[row1:row2,col1:col2]
    feature_vector = np.concatenate((feature_vector,Compute_histogram(img22)))

    img23 = img[row1:row2,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img23)))


    img3 = img[row2:row3+row2,:col1]
    feature_vector = np.concatenate((feature_vector,   Compute_histogram(img3)))
    img32 = img[row2:row3+row2,col1:col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img32)))

    img33 = img[row2:row3+row2,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img33)))

    return feature_vector

In [27]:
def Compute_Feature_vector(images):
    features = np.array([Compute_3x3_vector(images[0])])
    for x in range(1,len(images)):
        features = np.append(features, [Compute_3x3_vector(images[x])], axis=0)
    print(f'Shape: {features.shape}')
    return features


In [28]:
image_vector = load_image_and_resize_vector(df)

In [29]:
image_vector.shape

(61, 224, 224, 3)

In [30]:
feature_matrix = Compute_Feature_vector(image_vector)

Shape: (61, 6912)


                                                Pickling the data

In [31]:
file_name = 'Compressed_data'

with open(file_name, 'wb') as handle:
    pkl.dump(feature_matrix, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [32]:
with open(file_name, 'rb') as handle:
    feature_matrix = pkl.load(handle)
print(feature_matrix.shape)

(61, 6912)
